<a href="https://colab.research.google.com/github/malowana/NLP_budzet_obywatelski/blob/main/BO_analiza_morf_LDA_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Analiza LDA dla budżetów obywatelskich, zgrupwanie nazw podstawowych per nazwa budżetu

In [10]:
import warnings
warnings.filterwarnings('ignore')
warnings.filterwarnings('ignore', 'SelectableGroups dict interface')

In [11]:
import pandas as pd
import numpy as np
import gensim
from gensim import corpora, models
from gensim.utils import simple_preprocess


In [12]:
# Wyświetlanie szerszych kolumn, aby nie skracać wyświetlanych nazw tematów
pd.set_option('max_colwidth', 400)

## Warszawa

In [49]:
df_waw = pd.read_csv('/content/drive/My Drive/Konkursy Kaggle/Budżet obywatelski/Morfeusz_WAW.csv')
df_waw.head()

,rok,miasto,nazwa,start_ind,end_ind,form,lemma,tag,qualifiers,_
0,2018,WARSZAWA,Wolskie lato filmowe,0,1,Wolskie,wolski,adj:pl:acc:m2.m3.f.n:pos,[],[]
1,2018,WARSZAWA,Wolskie lato filmowe,0,1,Wolskie,wolski,adj:pl:nom.voc:m2.m3.f.n:pos,[],[]
2,2018,WARSZAWA,Wolskie lato filmowe,0,1,Wolskie,wolski,adj:sg:acc:n:pos,[],[]
3,2018,WARSZAWA,Wolskie lato filmowe,0,1,Wolskie,wolski,adj:sg:nom.voc:n:pos,[],[]
4,2018,WARSZAWA,Wolskie lato filmowe,0,1,Wolskie,Wolskie,subst:pl:nom.acc.voc:n:pt,['nazwa_geograficzna'],[]


In [50]:
#wyciagniecie pierwszej wartosci z kolumny lemma (przed :)
df_waw['lemma_clean'] = df_waw['lemma'].str.split(':', expand=True)[0]
df_waw.head()

,rok,miasto,nazwa,start_ind,end_ind,form,lemma,tag,qualifiers,_,lemma_clean
0,2018,WARSZAWA,Wolskie lato filmowe,0,1,Wolskie,wolski,adj:pl:acc:m2.m3.f.n:pos,[],[],wolski
1,2018,WARSZAWA,Wolskie lato filmowe,0,1,Wolskie,wolski,adj:pl:nom.voc:m2.m3.f.n:pos,[],[],wolski
2,2018,WARSZAWA,Wolskie lato filmowe,0,1,Wolskie,wolski,adj:sg:acc:n:pos,[],[],wolski
3,2018,WARSZAWA,Wolskie lato filmowe,0,1,Wolskie,wolski,adj:sg:nom.voc:n:pos,[],[],wolski
4,2018,WARSZAWA,Wolskie lato filmowe,0,1,Wolskie,Wolskie,subst:pl:nom.acc.voc:n:pt,['nazwa_geograficzna'],[],Wolskie


In [51]:
#usuniecie slow z poszczegolnymi tagami jak spojniki itp
tags = ['prep', 'conj', 'interj', 'part', 'adv']
df_waw_clean = df_waw[~df_waw['tag'].str.contains('|'.join(tags))]
df_waw_clean.sample(10)

,rok,miasto,nazwa,start_ind,end_ind,form,lemma,tag,qualifiers,_,lemma_clean
43708,2019,WARSZAWA,Chodniki zamiast błota - poprawa dojścia do metra i przystanku kolejowego przy Stadionie Narodowym,9,10,przystanku,przystanek,subst:sg:voc:m3,['nazwa_pospolita'],[],przystanek
50281,2019,WARSZAWA,Koło Kulinarne „Jem zdrowo”. Warsztaty gotowania dla dzieci na Chomiczówce,7,8,Warsztaty,warsztat,subst:pl:nom.acc.voc:m3,['nazwa_pospolita'],[],warsztat
29652,2018,WARSZAWA,Renesans i barok w tańcu - warsztaty tańca historycznego w Aninie,4,5,tańcu,taniec,subst:sg:loc:m3,['nazwa_pospolita'],[],taniec
64185,2020,WARSZAWA,"Dziecięce centrum rozwoju - Zielona Białołęka - bezpłatne zajęcia specjalistyczne dla naszych dzieci (logopedia, terapia pedagogiczna, psychologia, gimnastyka korekcyjna, diagnoza SI)",3,4,-,-,interp,[],[],-
74444,2020,WARSZAWA,Działalność edukacyjno-kulturalna dla dzieci i młodzieży oraz dorosłych w bibliotekach na Pradze-Południe,14,15,-,-,interp,[],[],-
55033,2019,WARSZAWA,"Potańcówki na dechach z muzyką na żywo w Domu Kultury „Rembertów""",8,9,Domu,dom,subst:sg:voc:m3,['nazwa_pospolita'],[],dom
67326,2020,WARSZAWA,Wygodna droga do Parku Młocińskiego - remont ul. Papirusów,8,9,.,.,interp,[],[],.
29086,2018,WARSZAWA,Letnie Plenery Malarskie,2,3,Malarskie,malarski,adj:sg:acc:n:pos,[],[],malarski
74581,2020,WARSZAWA,Ćwiczę z mamą i tatą - sportowe soboty. Kocham Pragę,9,10,Kocham,kochać,fin:sg:pri:imperf,[],[],kochać
24483,2018,WARSZAWA,Sadyba filmowa – kino plenerowe,2,3,–,–,interp,[],[],–


In [52]:
#zgrupwanie poszczegolnych slow per nazwa
df_waw1 = df_waw_clean.pivot_table(index=['nazwa'], values=['lemma_clean'], aggfunc=lambda x: ', '.join(x.astype(str)))
df_waw1.head()

,lemma_clean
nazwa,
Budowa chodnika pod skarpą – w rejonie ulicy Rydygiera wzdłuż wiaduktu ulicy Jana Pawła II,"budowa, Budowo, Budowo, chodnik, skarpa, –, rejon, rejon, ulica, ulica, Rydygier, wzdłużyć, wiadukt, ulica, ulica, Jana, Jana, Jan, paweł, Pawła, Paweł, II"
Funkcjonalne stojaki na deskorolki z bezpiecznym zabezpieczeniem dla wszystkich ursynowskich szkół,"funkcjonalny, funkcjonalny, funkcjonalny, funkcjonalny, stojak, deskorolka, deskorolka, bezpieczny, bezpieczny, bezpieczny, zabezpieczenie, zabezpieczyć, wszyscy, wszyscy, wszyscy, wszystek, wszystek, wszystek, ursynowski, ursynowski, ursynowski, szkoła"
Jestem aktywny i mogę zmienić świat – czyli warsztaty dla zdobywców świata (warsztaty rozwojowe dla młodzieży 16-24),"być, aktywny, aktywny, móc, zmienić, świat, –, warsztaty, warsztat, zdobywca, świat, (, warsztaty, warsztat, rozwojowy, rozwojowy, rozwojowy, rozwojowy, młodzież, młodzież, młodzież, młodzież, 16, -, 24, )"
Mali wielcy Twórcy-warsztaty teatralne dla dzieci i młodzieży,"Mali, mały, Mały, mały, wielki, wielki, twórca, twórca, twórca, -, warsztaty, warsztat, teatralny, teatralny, teatralny, teatralny, dziecko, dziecko, młodzież, młodzież, młodzież, młodzież"
PING-PONG 8+ nauka gry od podstaw dla dzieci i młodzieży z dzielnic Bemowo i Wola.,"ping-pong, 8, +, nauka, gra, gra, oda, podstawa, podstawić, dziecko, dziecko, młodzież, młodzież, młodzież, młodzież, dzielnica, Bemowo, wola, Wola, Wola, Wola, wole, wole, woli, ."


In [53]:
#preprocessing i stworzenie tokenów
df_waw1['tokens'] = df_waw1['lemma_clean'].apply(simple_preprocess)
df_waw1.sample(10)

,lemma_clean,tokens
nazwa,,
Butelkomaty na Targówku,"Butelkomaty, Targówek, Targówek","[butelkomaty, targówek, targówek]"
Kurs pierwszej pomocy pediatrycznej dla dzieci,"kurs, Kurs, Kurs, Kursa, pierwszy, pierwszy, pierwszy, pierwsza, pierwsza, pomoc, pomoc, pomoc, pomoc, pediatryczny, pediatryczny, pediatryczny, dziecko, dziecko","[kurs, kurs, kurs, kursa, pierwszy, pierwszy, pierwszy, pierwsza, pierwsza, pomoc, pomoc, pomoc, pomoc, pediatryczny, pediatryczny, pediatryczny, dziecko, dziecko]"
Koparka dla dzieci w Jordanku,"koparka, dziecko, dziecko, jordanek, jordanek, Jordanek, Jordanek","[koparka, dziecko, dziecko, jordanek, jordanek, jordanek, jordanek]"
Przebudowa ścieżki oraz oświetlenia w ul. Abecadło i ul. Papirusów,"przebudowa, ścieżka, ścieżka, ścieżek, oświetlenie, oświetlenie, oświetlić, oświetlić, ulica, ul, ., abecadło, ulica, ul, ., papirus, papirus, papirus","[przebudowa, ścieżka, ścieżka, ścieżek, oświetlenie, oświetlenie, oświetlić, oświetlić, ulica, ul, abecadło, ulica, ul, papirus, papirus, papirus]"
"Poprawa bezpieczeństwa pieszych-budowa miejsc parkingowych na ul. Łukowskiej, od Komorskiej do Osieckiej, przy Osiedlu Ostrobramska","poprawa, Poprawa, Poprawa, bezpieczeństwo, bezpieczeństwo, pieszy, pieszy, -, budowa, miejsce, parkingowy, parkingowy, parkingowy, parkingowy, parkingowy, ulica, ul, ., łukowski, łukowski, łukowski, Łukowska, Łukowska, ,, oda, Komorska, Komorska, do, Osiecka, Osiecka, osiecki, osiecki, osiecki, ,, osiedle, osiedle, ostrobramski","[poprawa, poprawa, poprawa, bezpieczeństwo, bezpieczeństwo, pieszy, pieszy, budowa, miejsce, parkingowy, parkingowy, parkingowy, parkingowy, parkingowy, ulica, ul, łukowski, łukowski, łukowski, łukowska, łukowska, oda, komorska, komorska, do, osiecka, osiecka, osiecki, osiecki, osiecki, osiedle, osiedle, ostrobramski]"
Remont chodnika przy ul. Mehoffera,"remont, remonta, chodnik, ulica, ul, ., Mehoffer","[remont, remonta, chodnik, ulica, ul, mehoffer]"
Garaż dla rowerów w zrewitalizowanej hydroforni wraz z ogródkiem miododajnym,"garaż, rower, zrewitalizować, hydrofornia, hydrofornia, hydrofornia, wraz, ogródek, miododajny, miododajny, miododajny","[garaż, rower, zrewitalizować, hydrofornia, hydrofornia, hydrofornia, wraz, ogródek, miododajny, miododajny, miododajny]"
Konkurs literacki,"konkurs, literacki, literacki","[konkurs, literacki, literacki]"
Impreza kulturalna z targiem śniadaniowym,"impreza, kulturalny, targ, śniadaniowy, śniadaniowy, śniadaniowy","[impreza, kulturalny, targ, śniadaniowy, śniadaniowy, śniadaniowy]"


In [55]:
#usunięcie duplikatów z listy w kolumnie tokens
df_waw1["tokens_new"] = df_waw1.tokens.map(np.unique)

In [56]:
df_waw1.head()

,lemma_clean,tokens,tokens_new
nazwa,,,
Budowa chodnika pod skarpą – w rejonie ulicy Rydygiera wzdłuż wiaduktu ulicy Jana Pawła II,"budowa, Budowo, Budowo, chodnik, skarpa, –, rejon, rejon, ulica, ulica, Rydygier, wzdłużyć, wiadukt, ulica, ulica, Jana, Jana, Jan, paweł, Pawła, Paweł, II","[budowa, budowo, budowo, chodnik, skarpa, rejon, rejon, ulica, ulica, rydygier, wzdłużyć, wiadukt, ulica, ulica, jana, jana, jan, paweł, pawła, paweł, ii]","[budowa, budowo, chodnik, ii, jan, jana, paweł, pawła, rejon, rydygier, skarpa, ulica, wiadukt, wzdłużyć]"
Funkcjonalne stojaki na deskorolki z bezpiecznym zabezpieczeniem dla wszystkich ursynowskich szkół,"funkcjonalny, funkcjonalny, funkcjonalny, funkcjonalny, stojak, deskorolka, deskorolka, bezpieczny, bezpieczny, bezpieczny, zabezpieczenie, zabezpieczyć, wszyscy, wszyscy, wszyscy, wszystek, wszystek, wszystek, ursynowski, ursynowski, ursynowski, szkoła","[funkcjonalny, funkcjonalny, funkcjonalny, funkcjonalny, stojak, deskorolka, deskorolka, bezpieczny, bezpieczny, bezpieczny, zabezpieczenie, zabezpieczyć, wszyscy, wszyscy, wszyscy, wszystek, wszystek, wszystek, ursynowski, ursynowski, ursynowski, szkoła]","[bezpieczny, deskorolka, funkcjonalny, stojak, szkoła, ursynowski, wszyscy, wszystek, zabezpieczenie, zabezpieczyć]"
Jestem aktywny i mogę zmienić świat – czyli warsztaty dla zdobywców świata (warsztaty rozwojowe dla młodzieży 16-24),"być, aktywny, aktywny, móc, zmienić, świat, –, warsztaty, warsztat, zdobywca, świat, (, warsztaty, warsztat, rozwojowy, rozwojowy, rozwojowy, rozwojowy, młodzież, młodzież, młodzież, młodzież, 16, -, 24, )","[być, aktywny, aktywny, móc, zmienić, świat, warsztaty, warsztat, zdobywca, świat, warsztaty, warsztat, rozwojowy, rozwojowy, rozwojowy, rozwojowy, młodzież, młodzież, młodzież, młodzież]","[aktywny, być, móc, młodzież, rozwojowy, warsztat, warsztaty, zdobywca, zmienić, świat]"
Mali wielcy Twórcy-warsztaty teatralne dla dzieci i młodzieży,"Mali, mały, Mały, mały, wielki, wielki, twórca, twórca, twórca, -, warsztaty, warsztat, teatralny, teatralny, teatralny, teatralny, dziecko, dziecko, młodzież, młodzież, młodzież, młodzież","[mali, mały, mały, mały, wielki, wielki, twórca, twórca, twórca, warsztaty, warsztat, teatralny, teatralny, teatralny, teatralny, dziecko, dziecko, młodzież, młodzież, młodzież, młodzież]","[dziecko, mali, mały, młodzież, teatralny, twórca, warsztat, warsztaty, wielki]"
PING-PONG 8+ nauka gry od podstaw dla dzieci i młodzieży z dzielnic Bemowo i Wola.,"ping-pong, 8, +, nauka, gra, gra, oda, podstawa, podstawić, dziecko, dziecko, młodzież, młodzież, młodzież, młodzież, dzielnica, Bemowo, wola, Wola, Wola, Wola, wole, wole, woli, .","[ping, pong, nauka, gra, gra, oda, podstawa, podstawić, dziecko, dziecko, młodzież, młodzież, młodzież, młodzież, dzielnica, bemowo, wola, wola, wola, wola, wole, wole, woli]","[bemowo, dziecko, dzielnica, gra, młodzież, nauka, oda, ping, podstawa, podstawić, pong, wola, wole, woli]"


In [59]:
#LDA tfidf
processed_docs = df_waw1['tokens_new']
vocab = gensim.corpora.Dictionary(processed_docs)
corpus = [vocab.doc2bow(doc) for doc in processed_docs]
tfidf = models.TfidfModel(corpus)
corpus_tfidf = tfidf[corpus]

lda_model = gensim.models.LdaMulticore(corpus_tfidf, num_topics=5, id2word=vocab, passes=2, workers=10)

for idx, topic in lda_model.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx, topic))

Topic: 0 
Words: 0.007*"dziecko" + 0.005*"biblioteka" + 0.005*"nr" + 0.005*"numer" + 0.005*"ulica" + 0.004*"zajęcie" + 0.004*"zajęcia" + 0.004*"zająć" + 0.004*"ul" + 0.004*"szkoła"
Topic: 1 
Words: 0.006*"zabawa" + 0.006*"plac" + 0.006*"ulica" + 0.005*"zabawić" + 0.005*"ul" + 0.004*"skwer" + 0.004*"chodnik" + 0.003*"dziecko" + 0.003*"gra" + 0.003*"rewitalizacja"
Topic: 2 
Words: 0.004*"ulica" + 0.004*"do" + 0.004*"drzewo" + 0.004*"ul" + 0.003*"szkoła" + 0.003*"bezpieczny" + 0.003*"dziecko" + 0.003*"zielony" + 0.003*"spotkać" + 0.003*"spotkanie"
Topic: 3 
Words: 0.004*"ulica" + 0.004*"park" + 0.004*"dziecko" + 0.004*"ul" + 0.004*"ławka" + 0.004*"zabawa" + 0.003*"oświetlenie" + 0.003*"oświetlić" + 0.003*"plac" + 0.003*"zabawić"
Topic: 4 
Words: 0.006*"ulica" + 0.004*"nowy" + 0.004*"ul" + 0.004*"nowa" + 0.004*"zielony" + 0.003*"do" + 0.003*"mieszkaniec" + 0.003*"biblioteka" + 0.003*"park" + 0.003*"zajęcia"


In [60]:
#usunięcie słów, które nie wnoszą za dużo i tokenizacja na nowo
stopwords = ['do', 'wraz', 'oraz', 'ul', 'ulica', 'nr', 'numer', 'to', 'on', 'przy', 'dla', 'na', 'po','jak', 'ten', 'nie', 'od', 'oda', 'czyli']
df_waw_clean4 = df_waw_clean[~df_waw_clean['lemma'].str.contains('|'.join(stopwords))]
df_waw_clean4.sample(10)

,rok,miasto,nazwa,start_ind,end_ind,form,lemma,tag,qualifiers,_,lemma_clean
35798,2019,WARSZAWA,Nowe nasadzenia na Rakowcu,0,1,Nowe,Nowy:Sm1,depr:pl:nom.acc.voc:m2,['nazwisko'],[],Nowy
44628,2019,WARSZAWA,Nowe ławki w Parku Praskim – dla Pragi,4,5,Praskim,Praski,subst:sg:inst:m1,['nazwisko'],[],Praski
54214,2019,WARSZAWA,"Ogólnodostępne stoły do gry w szachy,",3,4,gry,gra,subst:sg:gen:f,['nazwa_pospolita'],[],gra
25334,2018,WARSZAWA,Remont placu zabaw przy ulicy Czerskiej,5,6,Czerskiej,Czerska,subst:sg:gen:f,['nazwisko'],[],Czerska
26608,2018,WARSZAWA,Podwieczorki Teatralne dla Dzieci na Komionkowskich Błoniach Elekcyjnych,1,2,Teatralne,teatralny,adj:pl:acc:m2.m3.f.n:pos,[],[],teatralny
35210,2019,WARSZAWA,"Drzewa i krzewy na ul. Strumykowej zamiast betonu, piachu i kurzu",12,13,kurzu,kurz,subst:sg:loc:m3,['nazwa_pospolita'],[],kurz
69097,2020,WARSZAWA,Nowa siłownia plenerowa w Parku Bartłomieja obok Szkoły Podstawowej nr 191 przy ul. Bokserskiej,0,1,Nowa,Nowa,subst:sg:voc:f,"['nazwa_geograficzna', 'nazwisko']",[],Nowa
32653,2019,WARSZAWA,Daszki - żagle dające cień na placach zabaw w parku Górczewska,7,8,zabaw,zabawić,impt:sg:sec:perf,[],[],zabawić
80714,2020,WARSZAWA,Powiększenie miejskiego placu zabaw wśród zieleni przy stacji metra Imielin,8,9,metra,metro,subst:sg:gen:n:ncol,['nazwa_pospolita'],[],metro
89395,2020,WARSZAWA,Miejsca parkingowe kiss and ride dla uczniów szkoły na Przasnyskiej,9,10,Przasnyskiej,Przasnyska,subst:sg:dat.loc:f,['nazwisko'],[],Przasnyska


In [61]:
df_waw2 = df_waw_clean4.pivot_table(index=['nazwa'], values=['lemma_clean'], aggfunc=lambda x: ', '.join(x.astype(str)))
df_waw2.head()

,lemma_clean
nazwa,
Budowa chodnika pod skarpą – w rejonie ulicy Rydygiera wzdłuż wiaduktu ulicy Jana Pawła II,"skarpa, –, Rydygier, wzdłużyć, wiadukt, Jan, paweł, Pawła, Paweł, II"
Funkcjonalne stojaki na deskorolki z bezpiecznym zabezpieczeniem dla wszystkich ursynowskich szkół,"deskorolka, deskorolka, bezpieczny, bezpieczny, bezpieczny, zabezpieczyć, wszyscy, wszyscy, wszyscy, wszystek, wszystek, wszystek, ursynowski, ursynowski, ursynowski, szkoła"
Jestem aktywny i mogę zmienić świat – czyli warsztaty dla zdobywców świata (warsztaty rozwojowe dla młodzieży 16-24),"być, aktywny, aktywny, móc, zmienić, świat, –, warsztaty, warsztat, świat, (, warsztaty, warsztat, rozwojowy, rozwojowy, rozwojowy, rozwojowy, 16, -, 24, )"
Mali wielcy Twórcy-warsztaty teatralne dla dzieci i młodzieży,"Mali, mały, Mały, mały, wielki, wielki, twórca, twórca, twórca, -, warsztaty, warsztat, teatralny, teatralny, teatralny, teatralny, dziecko, dziecko"
PING-PONG 8+ nauka gry od podstaw dla dzieci i młodzieży z dzielnic Bemowo i Wola.,"8, +, gra, gra, dziecko, dziecko, dzielnica, Bemowo, wola, Wola, Wola, Wola, wole, wole, woli, ."


In [62]:
df_waw2['tokens'] = df_waw2['lemma_clean'].apply(simple_preprocess)
df_waw2.sample(5)

,lemma_clean,tokens
nazwa,,
Rewitalizacja skweru przy ul. Glebowej oraz wycinka drzew,"rewitalizacja, skwer, ., glebowy, glebowy, glebowy, wycinka, wycinek, drzewo","[rewitalizacja, skwer, glebowy, glebowy, glebowy, wycinka, wycinek, drzewo]"
"Led-y na zebry - bezpieczne, doświetlone przejścia dla pieszych na ul. Sarmackiej","leda, Leda, -, y, zebra, zebra, -, bezpieczny, bezpieczny, bezpieczny, bezpieczny, ,, przejść, przejść, przejście, przejście, pieszy, pieszy, pieszy, pieszy, pieszy, ., sarmacki, sarmacki, sarmacki","[leda, leda, zebra, zebra, bezpieczny, bezpieczny, bezpieczny, bezpieczny, przejść, przejść, przejście, przejście, pieszy, pieszy, pieszy, pieszy, pieszy, sarmacki, sarmacki, sarmacki]"
Budowa ścieżki rowerowej i remont chodnika wzdłuż Domaniewskiej,"ścieżka, ścieżka, ścieżek, rowerowy, rowerowy, rowerowy, wzdłużyć","[ścieżka, ścieżka, ścieżek, rowerowy, rowerowy, rowerowy, wzdłużyć]"
Siłownia Plenerowa - Kazimierzowska Fitness,"siłownia, plenerowy, -, kazimierzowski, fitness, fitness","[siłownia, plenerowy, kazimierzowski, fitness, fitness]"
Zakup dodatkowego wyposażenia do biblioteki dziecięcej w nowej Kulturotece,"zakup, zakupić, biblioteka, biblioteka, dziecięcy, dziecięcy, dziecięcy, nowa, nowa, nowy, nowy, nowy","[zakup, zakupić, biblioteka, biblioteka, dziecięcy, dziecięcy, dziecięcy, nowa, nowa, nowy, nowy, nowy]"


In [63]:
df_waw2["tokens_new"] = df_waw2.tokens.map(np.unique)
df_waw2.head()

,lemma_clean,tokens,tokens_new
nazwa,,,
Budowa chodnika pod skarpą – w rejonie ulicy Rydygiera wzdłuż wiaduktu ulicy Jana Pawła II,"skarpa, –, Rydygier, wzdłużyć, wiadukt, Jan, paweł, Pawła, Paweł, II","[skarpa, rydygier, wzdłużyć, wiadukt, jan, paweł, pawła, paweł, ii]","[ii, jan, paweł, pawła, rydygier, skarpa, wiadukt, wzdłużyć]"
Funkcjonalne stojaki na deskorolki z bezpiecznym zabezpieczeniem dla wszystkich ursynowskich szkół,"deskorolka, deskorolka, bezpieczny, bezpieczny, bezpieczny, zabezpieczyć, wszyscy, wszyscy, wszyscy, wszystek, wszystek, wszystek, ursynowski, ursynowski, ursynowski, szkoła","[deskorolka, deskorolka, bezpieczny, bezpieczny, bezpieczny, zabezpieczyć, wszyscy, wszyscy, wszyscy, wszystek, wszystek, wszystek, ursynowski, ursynowski, ursynowski, szkoła]","[bezpieczny, deskorolka, szkoła, ursynowski, wszyscy, wszystek, zabezpieczyć]"
Jestem aktywny i mogę zmienić świat – czyli warsztaty dla zdobywców świata (warsztaty rozwojowe dla młodzieży 16-24),"być, aktywny, aktywny, móc, zmienić, świat, –, warsztaty, warsztat, świat, (, warsztaty, warsztat, rozwojowy, rozwojowy, rozwojowy, rozwojowy, 16, -, 24, )","[być, aktywny, aktywny, móc, zmienić, świat, warsztaty, warsztat, świat, warsztaty, warsztat, rozwojowy, rozwojowy, rozwojowy, rozwojowy]","[aktywny, być, móc, rozwojowy, warsztat, warsztaty, zmienić, świat]"
Mali wielcy Twórcy-warsztaty teatralne dla dzieci i młodzieży,"Mali, mały, Mały, mały, wielki, wielki, twórca, twórca, twórca, -, warsztaty, warsztat, teatralny, teatralny, teatralny, teatralny, dziecko, dziecko","[mali, mały, mały, mały, wielki, wielki, twórca, twórca, twórca, warsztaty, warsztat, teatralny, teatralny, teatralny, teatralny, dziecko, dziecko]","[dziecko, mali, mały, teatralny, twórca, warsztat, warsztaty, wielki]"
PING-PONG 8+ nauka gry od podstaw dla dzieci i młodzieży z dzielnic Bemowo i Wola.,"8, +, gra, gra, dziecko, dziecko, dzielnica, Bemowo, wola, Wola, Wola, Wola, wole, wole, woli, .","[gra, gra, dziecko, dziecko, dzielnica, bemowo, wola, wola, wola, wola, wole, wole, woli]","[bemowo, dziecko, dzielnica, gra, wola, wole, woli]"


In [64]:
#LDA tfidf
processed_docs = df_waw2['tokens_new']
vocab = gensim.corpora.Dictionary(processed_docs)
corpus = [vocab.doc2bow(doc) for doc in processed_docs]
tfidf = models.TfidfModel(corpus)
corpus_tfidf = tfidf[corpus]

lda_model = gensim.models.LdaMulticore(corpus_tfidf, num_topics=6, id2word=vocab, passes=2, workers=10)

for idx, topic in lda_model.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx, topic))

Topic: 0 
Words: 0.009*"park" + 0.009*"dziecko" + 0.007*"praga" + 0.006*"biblioteka" + 0.006*"bezpieczny" + 0.005*"zakup" + 0.005*"ławka" + 0.005*"zakupić" + 0.005*"mokotów" + 0.005*"zabawa"
Topic: 1 
Words: 0.007*"drzewo" + 0.007*"nowy" + 0.006*"nowa" + 0.006*"park" + 0.005*"zieleń" + 0.005*"zielenić" + 0.005*"miejsce" + 0.005*"biblioteka" + 0.004*"każdy" + 0.004*"drogi"
Topic: 2 
Words: 0.008*"dziecko" + 0.006*"pieszy" + 0.006*"zajęcie" + 0.006*"zajęcia" + 0.006*"zająć" + 0.005*"nowy" + 0.005*"nowa" + 0.004*"warsztaty" + 0.004*"warsztat" + 0.004*"nowe"
Topic: 3 
Words: 0.013*"plac" + 0.012*"zabawa" + 0.012*"zabawić" + 0.008*"plenerowy" + 0.007*"senior" + 0.006*"park" + 0.006*"siłownia" + 0.005*"warszawa" + 0.005*"rowerowy" + 0.005*"miejsce"
Topic: 4 
Words: 0.010*"skwer" + 0.009*"warsztat" + 0.009*"warsztaty" + 0.009*"zająć" + 0.009*"zajęcie" + 0.009*"zajęcia" + 0.007*"dziecko" + 0.005*"praga" + 0.005*"szkoła" + 0.004*"wzdłużyć"
Topic: 5 
Words: 0.007*"biblioteka" + 0.006*"dziecko" +

## Reszta miast

In [71]:
df_roc = pd.read_csv('/content/drive/My Drive/Konkursy Kaggle/Budżet obywatelski/Morfeusz_ROC.csv')
df_roc.head()

,rok,miasto,nazwa,start_ind,end_ind,form,lemma,tag,qualifiers,_
0,2020,GDYNIA,"Doposażenie parku pomiędzy Estakadą Kwiatkowskiego a wieżowcami przy ul. Dantyszka 2,4,8 w siłownię zewnętrzną i elementy małej architektury - pergole, bujane ławki itp.",0,1,Doposażenie,doposażyć,ger:sg:nom.acc:n:perf:aff,[],[]
1,2020,GDYNIA,"Doposażenie parku pomiędzy Estakadą Kwiatkowskiego a wieżowcami przy ul. Dantyszka 2,4,8 w siłownię zewnętrzną i elementy małej architektury - pergole, bujane ławki itp.",1,2,parku,park,subst:sg:gen:m3,['nazwa_pospolita'],[]
2,2020,GDYNIA,"Doposażenie parku pomiędzy Estakadą Kwiatkowskiego a wieżowcami przy ul. Dantyszka 2,4,8 w siłownię zewnętrzną i elementy małej architektury - pergole, bujane ławki itp.",1,2,parku,park,subst:sg:loc:m3,['nazwa_pospolita'],[]
3,2020,GDYNIA,"Doposażenie parku pomiędzy Estakadą Kwiatkowskiego a wieżowcami przy ul. Dantyszka 2,4,8 w siłownię zewnętrzną i elementy małej architektury - pergole, bujane ławki itp.",1,2,parku,park,subst:sg:voc:m3,['nazwa_pospolita'],[]
4,2020,GDYNIA,"Doposażenie parku pomiędzy Estakadą Kwiatkowskiego a wieżowcami przy ul. Dantyszka 2,4,8 w siłownię zewnętrzną i elementy małej architektury - pergole, bujane ławki itp.",2,3,pomiędzy,pomiędzy,prep:acc,[],[]


In [72]:
#wyciagniecie pierwszej wartosci z kolumny lemma (przed :)
df_roc['lemma_clean'] = df_roc['lemma'].str.split(':', expand=True)[0]
df_roc.head()

,rok,miasto,nazwa,start_ind,end_ind,form,lemma,tag,qualifiers,_,lemma_clean
0,2020,GDYNIA,"Doposażenie parku pomiędzy Estakadą Kwiatkowskiego a wieżowcami przy ul. Dantyszka 2,4,8 w siłownię zewnętrzną i elementy małej architektury - pergole, bujane ławki itp.",0,1,Doposażenie,doposażyć,ger:sg:nom.acc:n:perf:aff,[],[],doposażyć
1,2020,GDYNIA,"Doposażenie parku pomiędzy Estakadą Kwiatkowskiego a wieżowcami przy ul. Dantyszka 2,4,8 w siłownię zewnętrzną i elementy małej architektury - pergole, bujane ławki itp.",1,2,parku,park,subst:sg:gen:m3,['nazwa_pospolita'],[],park
2,2020,GDYNIA,"Doposażenie parku pomiędzy Estakadą Kwiatkowskiego a wieżowcami przy ul. Dantyszka 2,4,8 w siłownię zewnętrzną i elementy małej architektury - pergole, bujane ławki itp.",1,2,parku,park,subst:sg:loc:m3,['nazwa_pospolita'],[],park
3,2020,GDYNIA,"Doposażenie parku pomiędzy Estakadą Kwiatkowskiego a wieżowcami przy ul. Dantyszka 2,4,8 w siłownię zewnętrzną i elementy małej architektury - pergole, bujane ławki itp.",1,2,parku,park,subst:sg:voc:m3,['nazwa_pospolita'],[],park
4,2020,GDYNIA,"Doposażenie parku pomiędzy Estakadą Kwiatkowskiego a wieżowcami przy ul. Dantyszka 2,4,8 w siłownię zewnętrzną i elementy małej architektury - pergole, bujane ławki itp.",2,3,pomiędzy,pomiędzy,prep:acc,[],[],pomiędzy


In [73]:
#usuniecie slow z poszczegolnymi tagami jak spojniki itp
tags = ['prep', 'conj', 'interj', 'part', 'adv']
df_roc_clean = df_roc[~df_roc['tag'].str.contains('|'.join(tags))]
df_roc_clean.sample(10)

,rok,miasto,nazwa,start_ind,end_ind,form,lemma,tag,qualifiers,_,lemma_clean
302668,2018/19,ŁÓDŹ,Bieg na orientację po osiedlu Batorego.,4,5,osiedlu,osiedle,subst:sg:loc:n:ncol,['nazwa_pospolita'],[],osiedle
105276,2019,KATOWICE,M/15/VI - Któredy do - NaprawMY oznakowanie Ronda Zietka,0,1,M,metr,brev:npun,[],[],metr
262597,2018,CHEŁM,Przebudowa chodnika przy ul. Starościńskiej - strona przy sklepie meblowym Skorek od ul. Kolejowej do wysepki przy Szkole Podstawowej Nr 5,13,14,ul,ul,subst:sg:nom.acc:m3,['nazwa_pospolita'],[],ul
30949,2020,GDAŃSK,Bezpieczne łagodne przejścia przez ulicę,1,2,łagodne,łagodny,adj:sg:acc:n:pos,[],[],łagodny
419015,2020,KRAKÓW,Zacieniona pergola W Orgódku Jordanowskim,3,4,Orgódku,Orgódku,ign,[],[],Orgódku
320591,2019,ŁÓDŹ,Zajęcia fizyczne z elementami samoobrony poprawiające koordynację ruchową dla osiedla Nad Nerem.,12,13,.,.,interp,[],[],.
200611,2019,SZCZECIN,Sentymentalny Szczecin. Wspomnienia o zakładach odzieżowych Dana i Odra,7,8,Dana,Dan,subst:sg:gen.acc:m1,['imię'],[],Dan
274497,2018,ZAMOŚĆ,"Remont parkingu w rejonie budynków ul. Peowiaków 78, 80, BOB, działka nr 23/6",5,6,ul,ulica,brev:pun,[],[],ulica
51520,2020,SOPOT,13. Profesjonalne pole do gry w minigolfa na sopockich błoniach,9,10,sopockich,sopocki,adj:pl:acc:m1:pos,[],[],sopocki
80866,2019,GLIWICE,Remont nawierzchni chodnika na odcinku ok. 35 m wzdluz skweru przy filii nr 5 MBP od strony ul. Czapli.,9,10,wzdluz,wzdluz,ign,[],[],wzdluz


In [76]:
#zgrupwanie poszczegolnych slow per nazwa
df_roc1 = df_roc_clean.pivot_table(index=['nazwa'], values=['lemma_clean'], aggfunc=lambda x: ', '.join(x.astype(str)))
df_roc1.head()

,lemma_clean
nazwa,
"Wykonanie i wyposażenie placu zabaw dla dzieci młodszych przy Przedszkolu Miejskim nr 9 ""Bajkowa Kraina"" przy ul Blachnickiego","wykonanie, wykonać, wyposażyć, wyposażenie, plac, plac, plac, zabawić, zabawa, dziecko, dziecko, młodsza, młodsza, młody, młody, młody, przedszkole, przedszkole, Miejski, Miejski, Miejski, Miejska, miejski, miejski, miejski, nr, numer, 9, "", bajkowy, kraina, Kraina, Krain, "", ul, Blachnicki"
"Działania sportowe i edukacyjno-kulturalne skierowane do dzieci, młodzieży i seniorów","działanie, działanie, działać, działać, sportowy, sportowy, sportowy, sportowy, edukacyjny, -, kulturalny, kulturalny, kulturalny, kulturalny, skierować, skierować, do, dziecko, dziecko, ,, młodzież, młodzież, młodzież, młodzież, senior"
"""Bezpieczne Jaworzno – nowe parkingi, chodniki, oświetlenie w dzielnicach"" - szacunkowe koszty: 2 390 200,00 zł",""", bezpieczny, bezpieczny, bezpieczny, bezpieczny, Jaworzno, –, nowy, nowy, nowy, nowy, nowa, nowy, parking, ,, chodnik, ,, oświetlić, oświetlenie, dzielnica, "", -, szacunkowy, szacunkowy, szacunkowy, szacunkowy, koszt, , 2, 390, 200,00, złoty"
"""Bezpieczne ulice dla mieszkańców – progi zwalniające w dzielnicach Jaworzna"" - szacunkowe koszty: 200 000,00 zł",""", bezpieczny, bezpieczny, bezpieczny, bezpieczny, ulica, mieszkaniec, –, próg, zwalniać, zwalniać, dzielnica, Jaworzna, Jaworzna, Jaworzno, Jaworzno, "", -, szacunkowy, szacunkowy, szacunkowy, szacunkowy, koszt, , 200, 000,00, złoty"
"""Boisko piłkarskie z trawy syntetycznej u Króla Kazimierza przy SP 21 w Jeleniu - reaktywacja""-szacunkowe koszty: 1 000 000,00 zł",""", boisko, piłkarski, piłkarski, piłkarski, piłkarski, trawa, trawa, syntetyczny, syntetyczny, syntetyczny, król, Król, król, Kazimierz, Kazimierza, Kazimierz, Kazimierz, SP, SP, Szkoła_Podstawowa, 21, jeleń, jeleń, Jeleń, Jeleń, jeleń, jeleń, Jeleń, Jeleń, -, reaktywacja, "", -, szacunkowy, szacunkowy, szacunkowy, szacunkowy, koszt, , 1, 000, 000,00, złoty"


In [78]:
#preprocessing i stworzenie tokenów
df_roc1['tokens'] = df_roc1['lemma_clean'].apply(simple_preprocess)
#usunięcie duplikatów z listy w kolumnie tokens
df_roc1["tokens_new"] = df_roc1.tokens.map(np.unique)
df_roc1.sample(10)

,lemma_clean,tokens,tokens_new
nazwa,,,
Rewitalizacja Ogrodu Świętej Katarzyny,"rewitalizacja, ogród, święty, święty, święty, święta, święta, Święta, Święta, Katarzyn, Katarzyna, Katarzyna, rewitalizacja, ogród, święty, święty, święty, święta, święta, Święta, Święta, Katarzyn, Katarzyna, Katarzyna","[rewitalizacja, ogród, święty, święty, święty, święta, święta, święta, święta, katarzyn, katarzyna, katarzyna, rewitalizacja, ogród, święty, święty, święty, święta, święta, święta, święta, katarzyn, katarzyna, katarzyna]","[katarzyn, katarzyna, ogród, rewitalizacja, święta, święty]"
P0022 Modernizacja boiska Pawłów,"P0022, modernizacja, boisko, boisko, Pawłów, Pawłów, Pawłów, Pawłowo, Pawły, paweł, Paweł","[modernizacja, boisko, boisko, pawłów, pawłów, pawłów, pawłowo, pawły, paweł, paweł]","[boisko, modernizacja, paweł, pawłowo, pawły, pawłów]"
DARMOWY SERWIS ROWERÓW W KAŻDEJ DZIELNICY - zakup i montaż samoobsługowych stacji naprawy rowerów,"darmowy, darmowy, serwis, Serwis, rower, każdy, każdy, każdy, dzielnica, dzielnica, Dzielnica, Dzielnica, -, zakup, zakupić, montaż, samoobsługowy, samoobsługowy, samoobsługowy, stacja, stacja, stacja, naprawa, naprawa, rower","[darmowy, darmowy, serwis, serwis, rower, każdy, każdy, każdy, dzielnica, dzielnica, dzielnica, dzielnica, zakup, zakupić, montaż, samoobsługowy, samoobsługowy, samoobsługowy, stacja, stacja, stacja, naprawa, naprawa, rower]","[darmowy, dzielnica, każdy, montaż, naprawa, rower, samoobsługowy, serwis, stacja, zakup, zakupić]"
Tężnia Solankowa na Zielonej,"tężnia, solankowy, Zielona, Zielona, zielony, zielony, zielony","[tężnia, solankowy, zielona, zielona, zielony, zielony, zielony]","[solankowy, tężnia, zielona, zielony]"
Budowa siłowni przy zbiegu ulic Zgierska-Czereśniowa.,"budowa, Budowo, Budowo, siłownia, siłownia, siłownia, zbieg, zbieg, zbieg, zbieg, zbieg, ulica, zgierski, Zgierska, Zgierska, -, czereśniowy, .","[budowa, budowo, budowo, siłownia, siłownia, siłownia, zbieg, zbieg, zbieg, zbieg, zbieg, ulica, zgierski, zgierska, zgierska, czereśniowy]","[budowa, budowo, czereśniowy, siłownia, ulica, zbieg, zgierska, zgierski]"
Przyjazny ogród.,"przyjazny, przyjazny, Przyjazny, Przyjazny, ogród, .","[przyjazny, przyjazny, przyjazny, przyjazny, ogród]","[ogród, przyjazny]"
Pokonać korki - czasomierze.,"pokonać, korki, korka, korka, korek, -, czasomierz, .","[pokonać, korki, korka, korka, korek, czasomierz]","[czasomierz, korek, korka, korki, pokonać]"
138. XXV Międzynarodowych Olsztyńskich Dni Folkloru WARMIA'2021,"138, ., XXV, międzynarodowy, międzynarodowy, międzynarodowy, Olsztyński, Olsztyński, Olsztyńskie, Olsztyńskie, Olsztyńska, Olsztyńska, olsztyński, olsztyński, olsztyński, dzień, dzień, folklor, WARMIA'2021","[xxv, międzynarodowy, międzynarodowy, międzynarodowy, olsztyński, olsztyński, olsztyńskie, olsztyńskie, olsztyńska, olsztyńska, olsztyński, olsztyński, olsztyński, dzień, dzień, folklor, warmia]","[dzień, folklor, międzynarodowy, olsztyńska, olsztyński, olsztyńskie, warmia, xxv]"
Parkuj i chodź bezpiecznie - wymiana chodnika i miejsc parkingowych przy ul. Sierakowskiego.,"parkować, chodzić, -, wymiana, chodnik, miejsce, parkingowy, parkingowy, parkingowy, parkingowy, parkingowy, ulica, ul, ., Sierakowski, sierakowski, sierakowski, .","[parkować, chodzić, wymiana, chodnik, miejsce, parkingowy, parkingowy, parkingowy, parkingowy, parkingowy, ulica, ul, sierakowski, sierakowski, sierakowski]","[chodnik, chodzić, miejsce, parkingowy, parkować, sierakowski, ul, ulica, wymiana]"


In [80]:
#LDA tfidf
processed_docs = df_roc1['tokens_new']
vocab = gensim.corpora.Dictionary(processed_docs)
corpus = [vocab.doc2bow(doc) for doc in processed_docs]
tfidf = models.TfidfModel(corpus)
corpus_tfidf = tfidf[corpus]

lda_model = gensim.models.LdaMulticore(corpus_tfidf, num_topics=5, id2word=vocab, passes=2, workers=10)

for idx, topic in lda_model.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx, topic))

Topic: 0 
Words: 0.008*"ulica" + 0.007*"ul" + 0.005*"budowa" + 0.005*"budowo" + 0.005*"chodnik" + 0.004*"nr" + 0.004*"plac" + 0.004*"numer" + 0.004*"parking" + 0.004*"zabawa"
Topic: 1 
Words: 0.005*"ulica" + 0.004*"park" + 0.004*"ul" + 0.004*"budowa" + 0.004*"budowo" + 0.004*"numer" + 0.004*"osiedle" + 0.004*"dziecko" + 0.003*"boisko" + 0.003*"nr"
Topic: 2 
Words: 0.008*"zabawa" + 0.008*"zabawić" + 0.008*"plac" + 0.006*"ulica" + 0.006*"ul" + 0.004*"remont" + 0.004*"park" + 0.004*"do" + 0.004*"remonta" + 0.003*"droga"
Topic: 3 
Words: 0.004*"ulica" + 0.004*"ul" + 0.003*"osiedle" + 0.003*"do" + 0.003*"budowa" + 0.003*"zabawa" + 0.003*"chodnik" + 0.003*"budowo" + 0.003*"plac" + 0.003*"sp"
Topic: 4 
Words: 0.007*"ulica" + 0.006*"ul" + 0.005*"chodnik" + 0.004*"park" + 0.004*"numer" + 0.004*"do" + 0.004*"nr" + 0.004*"remont" + 0.004*"remonta" + 0.004*"miejski"


In [82]:
#usunięcie słów, które nie wnoszą za dużo i tokenizacja na nowo
stopwords = ['do', 'wraz', 'oraz', 'ul', 'ulica', 'nr', 'numer', 'to', 'on', 'przy', 'dla', 'na', 'po','jak', 'ten', 'nie', 'od', 'oda', 'czyli']
df_roc_clean4 = df_roc_clean[~df_roc_clean['lemma'].str.contains('|'.join(stopwords))]
df_roc2 = df_roc_clean4.pivot_table(index=['nazwa'], values=['lemma_clean'], aggfunc=lambda x: ', '.join(x.astype(str)))
df_roc2['tokens'] = df_roc2['lemma_clean'].apply(simple_preprocess)
df_roc2["tokens_new"] = df_roc2.tokens.map(np.unique)

In [84]:
#LDA tfidf
processed_docs = df_roc2['tokens_new']
vocab = gensim.corpora.Dictionary(processed_docs)
corpus = [vocab.doc2bow(doc) for doc in processed_docs]
tfidf = models.TfidfModel(corpus)
corpus_tfidf = tfidf[corpus]

lda_model = gensim.models.LdaMulticore(corpus_tfidf, num_topics=6, id2word=vocab, passes=2, workers=10)

for idx, topic in lda_model.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx, topic))

Topic: 0 
Words: 0.009*"parking" + 0.009*"bezpieczny" + 0.006*"zabawa" + 0.006*"plac" + 0.005*"zabawić" + 0.004*"teren" + 0.004*"wzdłużyć" + 0.004*"sp" + 0.004*"szkoła" + 0.004*"miejsce"
Topic: 1 
Words: 0.010*"park" + 0.008*"boisko" + 0.006*"teren" + 0.005*"przedszkole" + 0.005*"zabawa" + 0.005*"plac" + 0.004*"zabawić" + 0.004*"miejski" + 0.004*"osiedle" + 0.004*"droga"
Topic: 2 
Words: 0.013*"plac" + 0.013*"zabawić" + 0.013*"zabawa" + 0.005*"dziecko" + 0.005*"rowerowy" + 0.004*"teren" + 0.004*"ścieżka" + 0.004*"park" + 0.004*"szkoła" + 0.004*"osiedle"
Topic: 3 
Words: 0.007*"osiedle" + 0.006*"park" + 0.006*"pieszy" + 0.005*"osiedlowy" + 0.005*"dziecko" + 0.005*"plac" + 0.005*"przejść" + 0.005*"przejście" + 0.004*"rowerowy" + 0.004*"miejsce"
Topic: 4 
Words: 0.006*"miejski" + 0.006*"zakup" + 0.006*"zakupić" + 0.006*"szkoła" + 0.006*"plac" + 0.005*"biblioteka" + 0.005*"etap" + 0.005*"park" + 0.005*"zabawa" + 0.004*"zabawić"
Topic: 5 
Words: 0.007*"siłownia" + 0.007*"zabawa" + 0.006*"pl

## 2018

In [85]:
df_2018 = pd.read_csv('/content/drive/My Drive/Konkursy Kaggle/Budżet obywatelski/Morfeusz_2018.csv')
df_2018.head()

,rok,miasto,nazwa,start_ind,end_ind,form,lemma,tag,qualifiers,_
0,2018,GDYNIA,"Zagospodarowanie terenu po bytym baraku przy ulicy Dedala - stworzenie sciezek, ustawienie fawek, nasadzenia zieleni i aranzacja matej architektury",0,1,Zagospodarowanie,zagospodarować,ger:sg:nom.acc:n:perf:aff,[],[]
1,2018,GDYNIA,"Zagospodarowanie terenu po bytym baraku przy ulicy Dedala - stworzenie sciezek, ustawienie fawek, nasadzenia zieleni i aranzacja matej architektury",1,2,terenu,teren,subst:sg:gen:m3,['nazwa_pospolita'],[]
2,2018,GDYNIA,"Zagospodarowanie terenu po bytym baraku przy ulicy Dedala - stworzenie sciezek, ustawienie fawek, nasadzenia zieleni i aranzacja matej architektury",2,3,po,po,prep:acc,[],[]
3,2018,GDYNIA,"Zagospodarowanie terenu po bytym baraku przy ulicy Dedala - stworzenie sciezek, ustawienie fawek, nasadzenia zieleni i aranzacja matej architektury",2,3,po,po,prep:dat,[],[]
4,2018,GDYNIA,"Zagospodarowanie terenu po bytym baraku przy ulicy Dedala - stworzenie sciezek, ustawienie fawek, nasadzenia zieleni i aranzacja matej architektury",2,3,po,po,prep:loc,[],[]


In [86]:
#wyciagniecie pierwszej wartosci z kolumny lemma (przed :)
df_2018['lemma_clean'] = df_2018['lemma'].str.split(':', expand=True)[0]
#usuniecie slow z poszczegolnymi tagami jak spojniki itp
tags = ['prep', 'conj', 'interj', 'part', 'adv']
df_2018_clean = df_2018[~df_2018['tag'].str.contains('|'.join(tags))]
#zgrupwanie poszczegolnych slow per nazwa
df_2018_1 = df_2018_clean.pivot_table(index=['nazwa'], values=['lemma_clean'], aggfunc=lambda x: ', '.join(x.astype(str)))
#preprocessing i stworzenie tokenów
df_2018_1['tokens'] = df_2018_1['lemma_clean'].apply(simple_preprocess)
#usunięcie duplikatów z listy w kolumnie tokens
df_2018_1["tokens_new"] = df_2018_1.tokens.map(np.unique)

In [87]:
#LDA tfidf
processed_docs = df_2018_1['tokens_new']
vocab = gensim.corpora.Dictionary(processed_docs)
corpus = [vocab.doc2bow(doc) for doc in processed_docs]
tfidf = models.TfidfModel(corpus)
corpus_tfidf = tfidf[corpus]

lda_model = gensim.models.LdaMulticore(corpus_tfidf, num_topics=6, id2word=vocab, passes=2, workers=10)

for idx, topic in lda_model.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx, topic))

Topic: 0 
Words: 0.006*"ulica" + 0.005*"ul" + 0.004*"zielony" + 0.004*"remont" + 0.004*"remonta" + 0.003*"do" + 0.003*"numer" + 0.003*"nr" + 0.003*"plac" + 0.003*"bezpieczny"
Topic: 1 
Words: 0.006*"nr" + 0.006*"numer" + 0.005*"ulica" + 0.005*"ul" + 0.004*"miejski" + 0.004*"szkoła" + 0.004*"do" + 0.003*"park" + 0.003*"boisko" + 0.003*"budowa"
Topic: 2 
Words: 0.006*"plac" + 0.005*"zabawa" + 0.005*"zabawić" + 0.004*"ulica" + 0.004*"park" + 0.004*"ul" + 0.004*"sp" + 0.003*"budowa" + 0.003*"do" + 0.003*"dziecko"
Topic: 3 
Words: 0.005*"ulica" + 0.005*"ul" + 0.004*"zabawić" + 0.004*"zabawa" + 0.004*"plac" + 0.003*"budowa" + 0.003*"do" + 0.002*"siłownia" + 0.002*"park" + 0.002*"dzielnica"
Topic: 4 
Words: 0.006*"ulica" + 0.006*"chodnik" + 0.005*"ul" + 0.005*"budowa" + 0.005*"plac" + 0.005*"budowo" + 0.004*"nowy" + 0.004*"zabawa" + 0.004*"zabawić" + 0.004*"miejsce"
Topic: 5 
Words: 0.006*"ulica" + 0.005*"ul" + 0.004*"do" + 0.004*"chodnik" + 0.004*"plac" + 0.004*"zakupić" + 0.004*"zakup" + 0.

In [90]:
#usunięcie słów, które nie wnoszą za dużo i tokenizacja na nowo
stopwords = ['do', 'wraz', 'oraz', 'ul', 'ulica', 'nr', 'numer', 'to', 'on', 'przy', 'dla', 'na', 'po','jak', 'ten', 'nie', 'od', 'oda', 'czyli']
df_2018_clean4 = df_2018_clean[~df_2018_clean['lemma'].str.contains('|'.join(stopwords))]
df_2018_2 = df_2018_clean4.pivot_table(index=['nazwa'], values=['lemma_clean'], aggfunc=lambda x: ', '.join(x.astype(str)))
df_2018_2['tokens'] = df_2018_2['lemma_clean'].apply(simple_preprocess)
df_2018_2["tokens_new"] = df_2018_2.tokens.map(np.unique)

In [91]:
#LDA tfidf
processed_docs = df_2018_2['tokens_new']
vocab = gensim.corpora.Dictionary(processed_docs)
corpus = [vocab.doc2bow(doc) for doc in processed_docs]
tfidf = models.TfidfModel(corpus)
corpus_tfidf = tfidf[corpus]

lda_model = gensim.models.LdaMulticore(corpus_tfidf, num_topics=6, id2word=vocab, passes=2, workers=10)

for idx, topic in lda_model.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx, topic))

Topic: 0 
Words: 0.005*"park" + 0.004*"centrum" + 0.004*"nowy" + 0.004*"miejsce" + 0.004*"nowa" + 0.004*"osiedle" + 0.004*"miejski" + 0.004*"zabawa" + 0.004*"teren" + 0.003*"plac"
Topic: 1 
Words: 0.006*"miejsce" + 0.005*"biblioteka" + 0.005*"dziecko" + 0.005*"zakup" + 0.005*"zakupić" + 0.004*"boisko" + 0.004*"osiedle" + 0.003*"park" + 0.003*"nowy" + 0.003*"szkoła"
Topic: 2 
Words: 0.009*"park" + 0.005*"parkingowy" + 0.005*"miejsce" + 0.005*"siłownia" + 0.004*"plenerowy" + 0.004*"szkoła" + 0.004*"dziecko" + 0.003*"rekreacyjny" + 0.003*"nowy" + 0.003*"nowa"
Topic: 3 
Words: 0.007*"plac" + 0.006*"bezpieczny" + 0.006*"zabawa" + 0.006*"sp" + 0.006*"zabawić" + 0.005*"biblioteka" + 0.004*"siłownia" + 0.004*"park" + 0.004*"boisko" + 0.004*"miejski"
Topic: 4 
Words: 0.017*"plac" + 0.016*"zabawić" + 0.016*"zabawa" + 0.008*"rowerowy" + 0.006*"wzdłużyć" + 0.005*"teren" + 0.005*"szkoła" + 0.005*"ścieżka" + 0.005*"przedszkole" + 0.005*"ścieżek"
Topic: 5 
Words: 0.007*"osiedle" + 0.005*"szkoła" + 0.

## 2019

In [92]:
df_2019 = pd.read_csv('/content/drive/My Drive/Konkursy Kaggle/Budżet obywatelski/Morfeusz_2019.csv')
df_2019.head()

,rok,miasto,nazwa,start_ind,end_ind,form,lemma,tag,qualifiers,_
0,2019,GDYNIA,"Parki kieszonkowe W 9 dzielnicach: Dqbrowa, Grabówek, Wzgórze Swietego Maksymiliana, Chylonia, Pogórze, Dziatki Lesne, Leszczynki, Pustki Cisowskie, Wielki Kack",0,1,Parki,park,subst:pl:nom.acc.voc:m3,['nazwa_pospolita'],[]
1,2019,GDYNIA,"Parki kieszonkowe W 9 dzielnicach: Dqbrowa, Grabówek, Wzgórze Swietego Maksymiliana, Chylonia, Pogórze, Dziatki Lesne, Leszczynki, Pustki Cisowskie, Wielki Kack",0,1,Parki,parka,subst:sg:gen:f,['nazwa_pospolita'],[]
2,2019,GDYNIA,"Parki kieszonkowe W 9 dzielnicach: Dqbrowa, Grabówek, Wzgórze Swietego Maksymiliana, Chylonia, Pogórze, Dziatki Lesne, Leszczynki, Pustki Cisowskie, Wielki Kack",0,1,Parki,parka,subst:pl:nom.acc.voc:f,['nazwa_pospolita'],[]
3,2019,GDYNIA,"Parki kieszonkowe W 9 dzielnicach: Dqbrowa, Grabówek, Wzgórze Swietego Maksymiliana, Chylonia, Pogórze, Dziatki Lesne, Leszczynki, Pustki Cisowskie, Wielki Kack",0,1,Parki,Parka:Sf,subst:sg:gen:f,"['imię', 'nazwisko']",[]
4,2019,GDYNIA,"Parki kieszonkowe W 9 dzielnicach: Dqbrowa, Grabówek, Wzgórze Swietego Maksymiliana, Chylonia, Pogórze, Dziatki Lesne, Leszczynki, Pustki Cisowskie, Wielki Kack",0,1,Parki,Parka:Sf,subst:pl:nom.acc.voc:f,"['imię', 'nazwisko']",[]


In [93]:
#wyciagniecie pierwszej wartosci z kolumny lemma (przed :)
df_2019['lemma_clean'] = df_2019['lemma'].str.split(':', expand=True)[0]
#usuniecie slow z poszczegolnymi tagami jak spojniki itp
tags = ['prep', 'conj', 'interj', 'part', 'adv']
df_2019_clean = df_2019[~df_2019['tag'].str.contains('|'.join(tags))]
#zgrupwanie poszczegolnych slow per nazwa
df_2019_1 = df_2019_clean.pivot_table(index=['nazwa'], values=['lemma_clean'], aggfunc=lambda x: ', '.join(x.astype(str)))
#preprocessing i stworzenie tokenów
df_2019_1['tokens'] = df_2019_1['lemma_clean'].apply(simple_preprocess)
#usunięcie duplikatów z listy w kolumnie tokens
df_2019_1["tokens_new"] = df_2019_1.tokens.map(np.unique)

In [94]:
#LDA tfidf
processed_docs = df_2019_1['tokens_new']
vocab = gensim.corpora.Dictionary(processed_docs)
corpus = [vocab.doc2bow(doc) for doc in processed_docs]
tfidf = models.TfidfModel(corpus)
corpus_tfidf = tfidf[corpus]

lda_model = gensim.models.LdaMulticore(corpus_tfidf, num_topics=6, id2word=vocab, passes=2, workers=10)

for idx, topic in lda_model.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx, topic))

Topic: 0 
Words: 0.007*"ulica" + 0.006*"ul" + 0.005*"numer" + 0.005*"nr" + 0.005*"szkoła" + 0.004*"budowa" + 0.004*"budowo" + 0.003*"rowerowy" + 0.003*"do" + 0.003*"bezpieczny"
Topic: 1 
Words: 0.005*"ulica" + 0.005*"ul" + 0.004*"plac" + 0.004*"zabawa" + 0.004*"zabawić" + 0.003*"park" + 0.003*"sportowy" + 0.003*"remont" + 0.003*"remonta" + 0.003*"dziecko"
Topic: 2 
Words: 0.005*"plac" + 0.005*"zabawa" + 0.005*"zabawić" + 0.004*"ulica" + 0.004*"park" + 0.004*"ul" + 0.003*"do" + 0.003*"remonta" + 0.003*"remont" + 0.003*"modernizacja"
Topic: 3 
Words: 0.007*"zabawa" + 0.007*"zabawić" + 0.007*"plac" + 0.004*"dziecko" + 0.004*"ulica" + 0.004*"ul" + 0.004*"nr" + 0.004*"chodnik" + 0.004*"numer" + 0.003*"park"
Topic: 4 
Words: 0.005*"ulica" + 0.005*"ul" + 0.004*"zielony" + 0.003*"do" + 0.003*"park" + 0.003*"budowa" + 0.003*"dziecko" + 0.003*"skwer" + 0.003*"budowo" + 0.003*"zielona"
Topic: 5 
Words: 0.006*"remont" + 0.006*"remonta" + 0.005*"chodnik" + 0.005*"ulica" + 0.004*"ul" + 0.004*"osiedl

In [95]:
#usunięcie słów, które nie wnoszą za dużo i tokenizacja na nowo
stopwords = ['do', 'wraz', 'oraz', 'ul', 'ulica', 'nr', 'numer', 'to', 'on', 'przy', 'dla', 'na', 'po','jak', 'ten', 'nie', 'od', 'oda', 'czyli']
df_2019_clean4 = df_2019_clean[~df_2019_clean['lemma'].str.contains('|'.join(stopwords))]
df_2019_2 = df_2019_clean4.pivot_table(index=['nazwa'], values=['lemma_clean'], aggfunc=lambda x: ', '.join(x.astype(str)))
df_2019_2['tokens'] = df_2019_2['lemma_clean'].apply(simple_preprocess)
df_2019_2["tokens_new"] = df_2019_2.tokens.map(np.unique)

In [96]:
#LDA tfidf
processed_docs = df_2019_2['tokens_new']
vocab = gensim.corpora.Dictionary(processed_docs)
corpus = [vocab.doc2bow(doc) for doc in processed_docs]
tfidf = models.TfidfModel(corpus)
corpus_tfidf = tfidf[corpus]

lda_model = gensim.models.LdaMulticore(corpus_tfidf, num_topics=6, id2word=vocab, passes=2, workers=10)

for idx, topic in lda_model.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx, topic))

Topic: 0 
Words: 0.008*"park" + 0.007*"boisko" + 0.006*"teren" + 0.005*"szkoła" + 0.005*"sp" + 0.005*"dziecko" + 0.004*"przestrzeń" + 0.004*"skwer" + 0.004*"przestrzenić" + 0.004*"plac"
Topic: 1 
Words: 0.008*"miejsce" + 0.004*"dzielnica" + 0.004*"nowy" + 0.004*"droga" + 0.004*"bezpieczny" + 0.004*"ogród" + 0.004*"nowa" + 0.003*"zakup" + 0.003*"drogi" + 0.003*"zakupić"
Topic: 2 
Words: 0.008*"miejski" + 0.006*"biblioteka" + 0.005*"sp" + 0.005*"zakup" + 0.005*"zakupić" + 0.005*"park" + 0.005*"miejska" + 0.005*"dziecko" + 0.005*"zajęcie" + 0.005*"zająć"
Topic: 3 
Words: 0.005*"senior" + 0.004*"droga" + 0.004*"drogi" + 0.004*"plac" + 0.004*"seniora" + 0.004*"rowerowy" + 0.004*"wzdłużyć" + 0.003*"zabawa" + 0.003*"park" + 0.003*"osiedlowy"
Topic: 4 
Words: 0.006*"bezpieczny" + 0.004*"park" + 0.004*"osiedle" + 0.003*"dziecko" + 0.003*"vi" + 0.003*"osoba" + 0.003*"teren" + 0.003*"dzielnica" + 0.003*"plac" + 0.003*"zabawa"
Topic: 5 
Words: 0.021*"zabawić" + 0.021*"zabawa" + 0.020*"plac" + 0.00

## 2020

In [97]:
df_2020 = pd.read_csv('/content/drive/My Drive/Konkursy Kaggle/Budżet obywatelski/Morfeusz_2020.csv')
df_2020.head()

,rok,miasto,nazwa,start_ind,end_ind,form,lemma,tag,qualifiers,_
0,2020,GDYNIA,"Doposażenie parku pomiędzy Estakadą Kwiatkowskiego a wieżowcami przy ul. Dantyszka 2,4,8 w siłownię zewnętrzną i elementy małej architektury - pergole, bujane ławki itp.",0,1,Doposażenie,doposażyć,ger:sg:nom.acc:n:perf:aff,[],[]
1,2020,GDYNIA,"Doposażenie parku pomiędzy Estakadą Kwiatkowskiego a wieżowcami przy ul. Dantyszka 2,4,8 w siłownię zewnętrzną i elementy małej architektury - pergole, bujane ławki itp.",1,2,parku,park,subst:sg:gen:m3,['nazwa_pospolita'],[]
2,2020,GDYNIA,"Doposażenie parku pomiędzy Estakadą Kwiatkowskiego a wieżowcami przy ul. Dantyszka 2,4,8 w siłownię zewnętrzną i elementy małej architektury - pergole, bujane ławki itp.",1,2,parku,park,subst:sg:loc:m3,['nazwa_pospolita'],[]
3,2020,GDYNIA,"Doposażenie parku pomiędzy Estakadą Kwiatkowskiego a wieżowcami przy ul. Dantyszka 2,4,8 w siłownię zewnętrzną i elementy małej architektury - pergole, bujane ławki itp.",1,2,parku,park,subst:sg:voc:m3,['nazwa_pospolita'],[]
4,2020,GDYNIA,"Doposażenie parku pomiędzy Estakadą Kwiatkowskiego a wieżowcami przy ul. Dantyszka 2,4,8 w siłownię zewnętrzną i elementy małej architektury - pergole, bujane ławki itp.",2,3,pomiędzy,pomiędzy,prep:acc,[],[]


In [98]:
#wyciagniecie pierwszej wartosci z kolumny lemma (przed :)
df_2020['lemma_clean'] = df_2020['lemma'].str.split(':', expand=True)[0]
#usuniecie slow z poszczegolnymi tagami jak spojniki itp
tags = ['prep', 'conj', 'interj', 'part', 'adv']
df_2020_clean = df_2020[~df_2020['tag'].str.contains('|'.join(tags))]
#zgrupwanie poszczegolnych slow per nazwa
df_2020_1 = df_2020_clean.pivot_table(index=['nazwa'], values=['lemma_clean'], aggfunc=lambda x: ', '.join(x.astype(str)))
#preprocessing i stworzenie tokenów
df_2020_1['tokens'] = df_2020_1['lemma_clean'].apply(simple_preprocess)
#usunięcie duplikatów z listy w kolumnie tokens
df_2020_1["tokens_new"] = df_2020_1.tokens.map(np.unique)

In [99]:
#LDA tfidf
processed_docs = df_2020_1['tokens_new']
vocab = gensim.corpora.Dictionary(processed_docs)
corpus = [vocab.doc2bow(doc) for doc in processed_docs]
tfidf = models.TfidfModel(corpus)
corpus_tfidf = tfidf[corpus]

lda_model = gensim.models.LdaMulticore(corpus_tfidf, num_topics=6, id2word=vocab, passes=2, workers=10)

for idx, topic in lda_model.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx, topic))

Topic: 0 
Words: 0.006*"ulica" + 0.005*"ul" + 0.004*"remont" + 0.004*"remonta" + 0.004*"zabawa" + 0.004*"plac" + 0.003*"zabawić" + 0.003*"park" + 0.003*"budowa" + 0.003*"budowo"
Topic: 1 
Words: 0.005*"ulica" + 0.004*"ul" + 0.004*"budowa" + 0.003*"budowo" + 0.003*"rowerowy" + 0.003*"chodnik" + 0.003*"park" + 0.002*"pieszy" + 0.002*"do" + 0.002*"ścieżka"
Topic: 2 
Words: 0.006*"ulica" + 0.005*"ul" + 0.004*"chodnik" + 0.004*"numer" + 0.004*"nr" + 0.004*"do" + 0.004*"szkoła" + 0.004*"budowa" + 0.004*"remonta" + 0.004*"remont"
Topic: 3 
Words: 0.007*"plac" + 0.007*"zabawić" + 0.007*"zabawa" + 0.005*"ulica" + 0.005*"ul" + 0.003*"park" + 0.003*"budowa" + 0.003*"bezpieczny" + 0.003*"biblioteka" + 0.003*"dziecko"
Topic: 4 
Words: 0.005*"ulica" + 0.004*"ul" + 0.004*"numer" + 0.004*"nr" + 0.004*"do" + 0.003*"zakup" + 0.003*"zakupić" + 0.003*"plac" + 0.003*"nowy" + 0.003*"sportowy"
Topic: 5 
Words: 0.005*"ulica" + 0.005*"ul" + 0.004*"chodnik" + 0.004*"zielony" + 0.004*"dziecko" + 0.003*"remont" +

In [101]:
#usunięcie słów, które nie wnoszą za dużo i tokenizacja na nowo
stopwords = ['do', 'wraz', 'oraz', 'ul', 'ulica', 'nr', 'numer', 'to', 'on', 'przy', 'dla', 'na', 'po','jak', 'ten', 'nie', 'od', 'oda', 'czyli']
df_2020_clean4 = df_2020_clean[~df_2020_clean['lemma'].str.contains('|'.join(stopwords))]
df_2020_2 = df_2020_clean4.pivot_table(index=['nazwa'], values=['lemma_clean'], aggfunc=lambda x: ', '.join(x.astype(str)))
df_2020_2['tokens'] = df_2020_2['lemma_clean'].apply(simple_preprocess)
df_2020_2["tokens_new"] = df_2020_2.tokens.map(np.unique)

In [102]:
#LDA tfidf
processed_docs = df_2020_2['tokens_new']
vocab = gensim.corpora.Dictionary(processed_docs)
corpus = [vocab.doc2bow(doc) for doc in processed_docs]
tfidf = models.TfidfModel(corpus)
corpus_tfidf = tfidf[corpus]

lda_model = gensim.models.LdaMulticore(corpus_tfidf, num_topics=6, id2word=vocab, passes=2, workers=10)

for idx, topic in lda_model.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx, topic))

Topic: 0 
Words: 0.007*"park" + 0.005*"plac" + 0.005*"zabawa" + 0.005*"osiedle" + 0.005*"zabawić" + 0.004*"osiedlowy" + 0.004*"nowy" + 0.003*"skwer" + 0.003*"biblioteka" + 0.003*"boisko"
Topic: 1 
Words: 0.013*"zabawa" + 0.012*"zabawić" + 0.012*"plac" + 0.007*"bezpieczny" + 0.007*"parking" + 0.005*"dziecko" + 0.005*"droga" + 0.004*"drogi" + 0.004*"zakupić" + 0.004*"zakup"
Topic: 2 
Words: 0.006*"wybieg" + 0.006*"pies" + 0.005*"teren" + 0.005*"zabawa" + 0.005*"plac" + 0.004*"boisko" + 0.004*"zabawić" + 0.004*"park" + 0.004*"miejski" + 0.004*"wniosek"
Topic: 3 
Words: 0.007*"park" + 0.006*"senior" + 0.005*"nowy" + 0.005*"bezpieczny" + 0.005*"nowa" + 0.005*"miejsce" + 0.005*"dziecko" + 0.004*"teren" + 0.004*"szkoła" + 0.004*"dzielnica"
Topic: 4 
Words: 0.005*"park" + 0.003*"rower" + 0.003*"street" + 0.003*"workout" + 0.003*"siłownia" + 0.003*"teren" + 0.003*"nowy" + 0.003*"rowerowy" + 0.003*"sp" + 0.003*"rekreacyjny"
Topic: 5 
Words: 0.005*"dziecko" + 0.005*"zajęcie" + 0.005*"zajęcia" + 0